In [1]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
sc = SparkContext('local')
spark = SparkSession(sc)

23/12/13 14:49:32 WARN Utils: Your hostname, krxps resolves to a loopback address: 127.0.1.1; using 192.168.68.61 instead (on interface wlp0s20f3)
23/12/13 14:49:32 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/12/13 14:49:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
df = spark.createDataFrame([(1, "a"), (2, "b")])

In [4]:
df.filter((col("_1") == 1) | (col("_2") == "a")).show()

+---+---+
| _1| _2|
+---+---+
|  1|  a|
+---+---+



In [32]:
(df
 .withColumn("test", lit("a_b"))
 .withColumn("yo", split(col("test"), "_")[1])
 .show()
)

+---+---+----+---+
| _1| _2|test| yo|
+---+---+----+---+
|  1|  a| a_b|  b|
|  2|  b| a_b|  b|
+---+---+----+---+



In [38]:
(df.withColumn("test", lit("a_b"))
 .withColumn("array", split("test", "_"))
 .withColumn("yo", explode(col("array")))
 .show()
)

+---+---+----+------+---+
| _1| _2|test| array| yo|
+---+---+----+------+---+
|  1|  a| a_b|[a, b]|  a|
|  1|  a| a_b|[a, b]|  b|
|  2|  b| a_b|[a, b]|  a|
|  2|  b| a_b|[a, b]|  b|
+---+---+----+------+---+



In [4]:
df.withColumn("upper", upper(col("_2"))).show()

+---+---+-----+
| _1| _2|upper|
+---+---+-----+
|  1|  a|    A|
|  2|  b|    B|
+---+---+-----+



In [40]:
(df.withColumn("description", lit("Description: bla bla"))
 .withColumn("result", regexp_replace("description", "^Description: ", ""))
 .show()
)

+---+---+--------------------+-------+
| _1| _2|         description| result|
+---+---+--------------------+-------+
|  1|  a|Description: bla bla|bla bla|
|  2|  b|Description: bla bla|bla bla|
+---+---+--------------------+-------+



In [43]:
(df.withColumnRenamed("_1", "id")
 .withColumn("name", col("_2"))
 .show()
)

+---+---+----+
| id| _2|name|
+---+---+----+
|  1|  a|   a|
|  2|  b|   b|
+---+---+----+



In [39]:
df_with_duplicates = df.withColumn("test", lit("yo"))
df_with_duplicates.show()

df_with_duplicates.drop_duplicates(subset = ["test"]).show()

+---+---+----+
| _1| _2|test|
+---+---+----+
|  1|  a|  yo|
|  2|  b|  yo|
+---+---+----+

+---+---+----+
| _1| _2|test|
+---+---+----+
|  1|  a|  yo|
+---+---+----+



In [45]:
df.agg(mean(col("_1")).alias("mean")).show()

+----+
|mean|
+----+
| 1.5|
+----+



In [40]:
df.agg(approx_count_distinct(col("_1"), 0.15).alias("distincCount")).show()

+------------+
|distincCount|
+------------+
|           2|
+------------+



In [57]:
assert df.orderBy("_1", ascending=False).select("_1").first()._1 == 2
assert df.sort(col("_1").desc()).first()._1 == 2 
assert df.sort(desc("_1")).first()._1 == 2


In [58]:
df.first()._1

1

In [8]:
df.describe("_1").show()

+-------+------------------+
|summary|                _1|
+-------+------------------+
|  count|                 2|
|   mean|               1.5|
| stddev|0.7071067811865476|
|    min|                 1|
|    max|                 2|
+-------+------------------+



In [72]:
integers = [1, 2, 2, 3, 4, 4, 5]
dfInt = spark.createDataFrame(integers, IntegerType())

assert [row.value for row in dfInt.sample(True, fraction=0.5, seed=3).collect()] == [4, 5]
assert [row.value for row in dfInt.sample(False, fraction=0.5, seed=3).collect()] == [1, 4, 4]

In [118]:
(
    df.withColumn("test", lit(1408024997).cast("timestamp"))
    .withColumn("month", month(col("test")))
    .withColumn("day_of_year", dayofyear(col("test")))
    .withColumn("yo", from_unixtime(lit(1408024997), "EEEE, MMM d, yyyy h:mm a"))
    .show(10, False)
)

+---+---+-------------------+-----+-----------+------------------------------+
|_1 |_2 |test               |month|day_of_year|yo                            |
+---+---+-------------------+-----+-----------+------------------------------+
|1  |a  |2014-08-14 16:03:17|8    |226        |Thursday, Aug 14, 2014 4:03 PM|
|2  |b  |2014-08-14 16:03:17|8    |226        |Thursday, Aug 14, 2014 4:03 PM|
+---+---+-------------------+-----+-----------+------------------------------+



In [31]:
(
    df.withColumn("test_na_1", lit(None).cast(StringType()))
    .withColumn("test_na_2", lit(None).cast(IntegerType()))
    .withColumn("test_na_3", lit(None).cast(IntegerType()))
    .withColumn("test_na_4", lit(None).cast(IntegerType()))
    .na.fill("yo")
    .na.fill({'test_na_2': 30})
    .na.fill(5, "test_na_4")
    .show()
)

+---+---+---------+---------+---------+---------+
| _1| _2|test_na_1|test_na_2|test_na_3|test_na_4|
+---+---+---------+---------+---------+---------+
|  1|  a|       yo|       30|     null|        5|
|  2|  b|       yo|       30|     null|        5|
+---+---+---------+---------+---------+---------+



In [126]:
a = spark.createDataFrame([("Alice", 1), ("Bob", 2), ("Rachid", 3)], "name:string, id: int")
b = spark.createDataFrame([("Franck", 3), ("Bernard", 4), ("Ramzy", 5)], "name:string, id: int")

a.crossJoin(b).show()
a.join(b, a.id == b.id).show()
a.join(b, ["id", "name"], "outer").show()

+------+---+-------+---+
|  name| id|   name| id|
+------+---+-------+---+
| Alice|  1| Franck|  3|
| Alice|  1|Bernard|  4|
| Alice|  1|  Ramzy|  5|
|   Bob|  2| Franck|  3|
|   Bob|  2|Bernard|  4|
|   Bob|  2|  Ramzy|  5|
|Rachid|  3| Franck|  3|
|Rachid|  3|Bernard|  4|
|Rachid|  3|  Ramzy|  5|
+------+---+-------+---+

+------+---+------+---+
|  name| id|  name| id|
+------+---+------+---+
|Rachid|  3|Franck|  3|
+------+---+------+---+

+---+-------+
| id|   name|
+---+-------+
|  1|  Alice|
|  2|    Bob|
|  3| Franck|
|  3| Rachid|
|  4|Bernard|
|  5|  Ramzy|
+---+-------+



In [25]:
df.withColumn("test", col("_1").cast(StringType())).printSchema()
df.withColumn("test", col("_1").cast("string")).printSchema()

root
 |-- _1: long (nullable = true)
 |-- _2: string (nullable = true)
 |-- test: string (nullable = true)

root
 |-- _1: long (nullable = true)
 |-- _2: string (nullable = true)
 |-- test: string (nullable = true)



In [75]:
def add42(n):
    return n + 42
    
df.createOrReplaceTempView("test")

spark.udf.register("ADD_42", add42)

spark.sql("select ADD_42(_1), _1 from test").show()

23/12/13 22:06:01 WARN SimpleFunctionRegistry: The function add_42 replaced a previously registered function.
+----------+---+
|ADD_42(_1)| _1|
+----------+---+
|        43|  1|
|        44|  2|
+----------+---+



In [76]:
def multiply_by_10(n):
    return n * 10

multiply_by_10_UDF = udf(multiply_by_10, IntegerType())

df.withColumn("test", multiply_by_10_UDF("_1")).show()

+---+---+----+
| _1| _2|test|
+---+---+----+
|  1|  a|  10|
|  2|  b|  20|
+---+---+----+



In [108]:
from pyspark import StorageLevel

# cache in MEMORY_AND_DISK_DESER (default)
df.cache().count()
assert str(df.storageLevel).startswith("Disk") == True
print(df.storageLevel)

df.unpersist()

# cache in MEMORY_ONLY
df.persist(StorageLevel.MEMORY_ONLY)
assert str(df.storageLevel).startswith("Memory") == True
print(df.storageLevel)

df.unpersist()

assert df.is_cached == False

Disk Memory Deserialized 1x Replicated
Memory Serialized 1x Replicated
